# WMT17

* To English
* Segment level data
* Scores
* Pearson correlation




To reproduce our results for a given metric import the metric config from the `configs.py` file and run the cells below.

E.g. for BERTScore:

`from geneval.replication.configs import bertscore_config as config`

In [1]:
# install dependencies

!pip -q install datasets
!pip -q install bert_score
!pip -q install git+https://github.com/google-research/bleurt.git
!pip -q install unbabel-comet
!pip -q install transformers
!pip -q install POT

In [2]:
!git clone https://github.com/drehero/geneval

fatal: destination path 'geneval' already exists and is not an empty directory.


In [3]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import pathlib

import datasets
import numpy as np
import pandas as pd
from scipy.stats import pearsonr

from geneval.geneval.data.wmt import WMT17

In [5]:
# import metric config

from geneval.replication.configs import moverscore_config as config

In [6]:
out_path = pathlib.Path(f"/content/drive/MyDrive/results/wmt17/")
lang_pairs = ["cs-en", "de-en", "fi-en", "lv-en", "ru-en", "tr-en", "zh-en"]

In [7]:
scorer = datasets.load_metric(config.metric_path, **config.load_args)

In [8]:
for lang_pair in lang_pairs:
    # load data
    wmt = WMT17(lang_pair)

    # compute score
    args = config.compute_args.copy()
    if config.uses_reference:
        args["references"] = wmt.references
    if config.uses_source:
        args["sources"] = wmt.sources
    
    scores = scorer.compute(
        predictions=wmt.translations,
        **args
    )

    # save
    df = pd.DataFrame({
        "translation": wmt.translations,
        "reference": wmt.references,
        "source": wmt.sources,
        "human_score": wmt.scores,
        "metric_score": scores[config.score_name] if config.score_name is not None else scores
    })
    if "model_type" in args.keys():
        fn = f"{lang_pair}-{args['model_type'].split('/')[-1]}.csv"
    elif "config_name" in config.load_args.keys():
        fn = f"{lang_pair}-{config.load_args['config_name'].split('/')[-1]}.csv"
    else:
        fn = f"{lang_pair}.csv"
    df.to_csv(out_path / config.metric_name / fn, index=False)

In [9]:
# load scores and compute pearson correlation
results = {}
for lang_pair in lang_pairs:
    if "model_type" in config.compute_args.keys():
        fn = f"{lang_pair}-{config.compute_args['model_type'].split('/')[-1]}.csv"
    elif "config_name" in config.load_args.keys():
        fn = f"{lang_pair}-{config.load_args['config_name'].split('/')[-1]}.csv"
    else:
        fn = f"{lang_pair}.csv"
    df = pd.read_csv(out_path / config.metric_name / fn)
    corr = pearsonr(df["metric_score"], df["human_score"])[0]
    results[lang_pair] = corr

In [10]:
results

{'cs-en': 0.602193531504667,
 'de-en': 0.6271585989729354,
 'fi-en': 0.7688493029415711,
 'lv-en': 0.6446894849157553,
 'ru-en': 0.6639949556937612,
 'tr-en': 0.7273878097748375,
 'zh-en': 0.6647566554514432}